Iniciando a uniação e validação das bases de dados extraidas

In [4]:
import pandas as pd

pd.set_option('display.max_columns', None)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

In [23]:
#Games 
df_list = []
for i in range(1, 8):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

# df_1 = pd.read_excel(path + r'\01_games.xlsx')
# df_2 = pd.read_excel(path + r'\02_games.xlsx')
# df_3 = pd.read_excel(path + r'\03_games.xlsx')
# df_4 = pd.read_excel(path + r'\04_games.xlsx')
# df_5 = pd.read_excel(path + r'\05_games.xlsx')
# df_6 = pd.read_excel(path + r'\06_games.xlsx')
# df_7 = pd.read_excel(path + r'\07_games.xlsx')

Visão geral de cada um das bases de dados, escolha dos principais atributos para avaliar público.

DF2 removido, não tem identificação explicita de partida, ano! da pra fazer mas as outras bases de dados já são suficientes.

In [6]:
# db_id = 3  # 0 - 6
# display(df_list[db_id].describe())
# display(df_list[db_id].head(-3))
# display(df_list[db_id].info())

Padronizar data, mandante e visitante para indexar arquivos e agrupalos

In [35]:
#fitragem série B comjumto 7
df_list[6] = df_list[6][df_list[6]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {
    'data': 'Data',
    'Date': 'Data',
    'data_partida': 'Data',
    'time_mandante': 'Mandante',
    'mandante': 'Mandante',
    'Home': 'Mandante',
    'time_visitante': 'Visitante',
    'visitante': 'Visitante',
    'Away': 'Visitante',
}
df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

# Padronizando Data
for i in range(len(df_list)):
    df_list[i]['Data'] = pd.to_datetime(df_list[i]['Data'], dayfirst=True, errors='coerce')
    df_list[i] = df_list[i].sort_values(by='Data').reset_index(drop=True)

mandante_list = []
visitante_list = []
for i in range(len(df_list)):
    mandante_list += df_list[i]['Mandante'].unique().tolist()
    visitante_list += df_list[i]['Visitante'].unique().tolist()
print(len(visitante_list))

298


In [3]:
## agrupamento de categorias de times com nomes diferentes mas que representam o mesmo time
## utilizando IA, e correção ortogrfica

import re
import unidecode
from spellchecker import SpellChecker
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Etapa 1: correção ortográfica
spell = SpellChecker(language='pt')

def corrigir_erros(texto):
    palavras = texto.split()
    corrigidas = [spell.correction(p) or p for p in palavras]
    return ' '.join(corrigidas)

# Etapa 2: padronização (sem acento, siglas, pontuação)
def padronizar_nome(nome):
    nome = corrigir_erros(nome)
    nome = unidecode.unidecode(nome.lower())
    nome = re.sub(r'\b(c\.?r\.?|clube de regatas|atl\.?|f\.?c\.?|mg|sp|rj|b\.?f\.?r\.?|s\.?p\.?f\.?c\.?|s\.?c\.?)\b', '', nome)
    nome = re.sub(r'[^a-z ]', '', nome)
    nome = re.sub(r'\s+', ' ', nome).strip()
    return nome


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\__init__.py)

In [ ]:

# Dados de exemplo
nomes = [
    "Flamengo", "CR Flamnego", "Clube de Regatas Flamengo", "Fla",
    "Atlétco-MG", "Atl. Mineiro", "Galo", "Atlético Mineiro",
    "Botafogo", "Btafogo", "BFR", "Fogão",
    "São Paulo", "SPFC", "Tricolor Paulista",
    "Vasco da Gama", "CR Vasco", "Vascão"
]

# Criação do DataFrame
df = pd.DataFrame(nomes, columns=["original"])
df['limpo'] = df['original'].apply(padronizar_nome)

# Etapa 3: embeddings com Sentence-BERT
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings = modelo.encode(df['limpo'].tolist())

# Função para encontrar os N nomes mais similares a uma dada consulta
def buscar_similares(consulta, embeddings, df, top_n=5):
    consulta_limpa = padronizar_nome(consulta)
    consulta_embedding = modelo.encode([consulta_limpa])[0]

    # Calcular a similaridade do cosseno entre a consulta e todos os nomes
    similaridades = cosine_similarity([consulta_embedding], embeddings)[0]

    # Obter os índices dos N nomes mais similares
    indices_similares = np.argsort(similaridades)[::-1][1:top_n+1] # Exclui o primeiro (a própria consulta se estiver na lista)

    # Criar um DataFrame com os resultados
    resultados = pd.DataFrame({
        'original': df['original'].iloc[indices_similares].tolist(),
        'similaridade': similaridades[indices_similares]
    })

    return resultados

# Exemplo de uso da função de busca semântica
consulta = "flamengo rj"
resultados = buscar_similares(consulta, embeddings, df, top_n=3)

print(f"Resultados da busca para '{consulta}':")
print(resultados)

consulta = "atletico mineiro futebol clube"
resultados = buscar_similares(consulta, embeddings, df, top_n=2)

print(f"\nResultados da busca para '{consulta}':")
print(resultados)

consulta = "bfr rio"
resultados = buscar_similares(consulta, embeddings, df, top_n=2)

print(f"\nResultados da busca para '{consulta}':")
print(resultados)

consulta = "sao paulo futebol"
resultados = buscar_similares(consulta, embeddings, df, top_n=2)

print(f"\nResultados da busca para '{consulta}':")
print(resultados)

consulta = "vasco"
resultados = buscar_similares(consulta, embeddings, df, top_n=2)

print(f"\nResultados da busca para '{consulta}':")
print(resultados)

🧹 1. Limpeza e padronização de texto
Ferramentas: spaCy, transformers, ChatGPT, regex, etc.

📌 Objetivo: remover ruídos (acentuação inconsistente, uso de maiúsculas/minúsculas, stopwords, espaços extras, etc.), padronizar dados para facilitar análise e comparação.


| **Técnica**                         | **O que faz**                                              | **Exemplo**                                              |
|-------------------------------------|-------------------------------------------------------------|----------------------------------------------------------|
| Conversão para minúsculas           | Uniformiza o texto, evita distinções desnecessárias         | "JOÃO da Silva" → "joão da silva"                       |
| Remoção de acentos e caracteres especiais | Facilita comparação e buscas                           | "ação" → "acao", "João!" → "joao"                      |
| Remoção de espaços extras           | Elimina espaços duplicados                                  | "João   Silva" → "João Silva"                          |
| Tokenização                         | Divide frases em palavras ou unidades                       | "Rua das Palmeiras" → ["Rua", "das", "Palmeiras"]         |
| Lematização                         | Reduz palavras à forma canônica (radical)                   | "andando" → "andar"                               |
| Remoção de stopwords                | Elimina palavras irrelevantes (ex: "de", "a", "por")        | "Rua de São João" → "Rua São João"                        |
| Padronização de abreviações         | Transforma formas abreviadas em completas                   | "Av." → "Avenida", "R." → "Rua"                          |
| Correção de erros comuns            | Corrige termos digitados incorretamente                     | "Avnda Paulista" → "Avenida Paulista"                    |
| Normalização semântica              | Usa NLP para unificar sinônimos/contextos usando GPT ou BERT                 | "Prof." → "Professor" ou “R. João Silva” → “Rua João Silva”                          |


In [26]:
# bibliotecas necessárias para limpeza e padronização
import re
import unidecode
import pandas as pd
import spacy

# inicializações
# Modelo SpaCy em português
nlp = spacy.load("pt_core_news_sm")

def para_minusculas(texto):
    if isinstance(texto, str):
        return texto.lower()
    return texto

def remover_acentos(texto):
    if isinstance(texto, str):
        return unidecode.unidecode(texto)
    return texto

def remover_caracteres_especiais(texto):
    if isinstance(texto, str):
        return re.sub(r'[^a-zA-Z0-9\s]', '', texto)
    return texto

def remover_espacos_extras(texto):
    if isinstance(texto, str):
        return re.sub(r'\s+', ' ', texto).strip()
    return texto

def remover_stopwords(texto):
    if isinstance(texto, str):
        doc = nlp(texto)
        return ' '.join([token.text for token in doc if not token.is_stop])
    return texto

def lematizar(texto):
    if isinstance(texto, str):
        doc = nlp(texto)
        return ' '.join([token.lemma_ for token in doc])
    return texto

def pipeline_avancado(texto, usar_chatgpt=False):
    texto = para_minusculas(texto)
    texto = remover_acentos(texto)
    texto = remover_caracteres_especiais(texto)
    texto = remover_espacos_extras(texto)

    # NLP com SpaCy
    texto = remover_stopwords(texto)
    texto = lematizar(texto)

    return texto

In [27]:
#display(df_list[0].head(1))
#display(df_list[0][['data', 'time_mandante', 'time_visitante']])

times_unicos = 0
for i in range(len(df_list)):
    #print(df_list[i]['Mandante'].unique())
    #print(set(df_list[i]['Mandante'].unique()) == set(df_list[i]['Visitante'].unique())) 
    #print(df_list[i]['Mandante'].unique(),df_list[i]['Visitante'].unique())
    times_unicos += len(df_list[i]['Mandante'].unique())

#display(df_list[2]['Mandante'].unique(),df_list[2]['Visitante'].unique())
    set1 = set(df_list[i]['Mandante'].unique())
    set2 = set(df_list[i]['Visitante'].unique())

    diferente = set1.symmetric_difference(set2)
    print(diferente)
print(times_unicos)
# ==============================
# Exemplo de uso com Pandas
# ==============================
# df['texto_limpo'] = df['coluna_original'].apply(lambda x: pipeline_avancado(x, usar_chatgpt=True))


set()
set()
{'Paysandy'}
set()
{'Mirassol'}
set()
set()
324


📝 2. Correção ortográfica
Ferramentas: pyspellchecker, modelos de linguagem (GPT, BERTimbau, etc.)

📌 Objetivo: corrigir erros de digitação ou ortografia que afetam a análise.

🔧 Técnicas:

Corretores simples baseados em dicionários: como o pyspellchecker, útil para textos curtos e termos conhecidos.

Modelos de linguagem (GPT, BERT): corrigem erros com base no contexto, muito úteis para frases completas ou nomes incomuns.

In [ ]:
# ==============================
# Correção ortográfica com Transformers
# ==============================
def corrigir_palavra_via_ia(texto):
    palavras = texto.split()
    texto_corrigido = []

    for palavra in palavras:
        entrada = texto.replace(palavra, preencher_mascara.tokenizer.mask_token)
        try:
            sugestao = preencher_mascara(entrada)[0]['token_str']
            texto_corrigido.append(sugestao)
        except:
            texto_corrigido.append(palavra)
    return ' '.join(texto_corrigido)

# ==============================
# Correção avançada com ChatGPT
# ==============================
def limpar_com_chatgpt(texto):
    prompt = f"""Revise o texto abaixo e corrija ortografia, acentuação e padronize nomes de ruas e abreviações. Retorne apenas o texto corrigido.
Texto: "{texto}"
Corrigido:"""
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=100
    )
    return response['choices'][0]['message']['content'].strip()

🧩 3. Preenchimento de valores ausentes (missing values)
Ferramentas: sklearn (KNNImputer, regressão), fancyimpute, modelos supervisionados.

📌 Objetivo: evitar perda de dados substituindo valores ausentes com estimativas coerentes.

🔧 Técnicas comuns:

KNNImputer: preenche com base em vizinhos mais próximos.

Regressão/MICE: usa relações entre variáveis para prever os valores faltantes.

Modelos supervisionados: predição usando modelos customizados (ex: árvore de decisão para prever “salário” com base em “idade”, “formação”, etc.).

🚨 4. Detecção de outliers
Ferramentas: Isolation Forest, DBSCAN, Autoencoders, Z-score, IQR.

📌 Objetivo: identificar e tratar valores anômalos que podem distorcer análises.

🔧 Técnicas:

Isolation Forest: modelo de árvore que isola outliers com base em caminhos curtos.

DBSCAN: algoritmo de cluster que separa ruído.

Autoencoders: redes neurais que aprendem padrões normais e destacam desvios.

Z-score/IQR: métodos estatísticos clássicos.

🔁 5. Deduplicação e fuzzy matching
Ferramentas: fuzzywuzzy, Levenshtein, sklearn (TF-IDF), Hugging Face models.

📌 Objetivo: detectar registros duplicados com variações pequenas (ex: “João da Silva” e “J. Silva”).

🔧 Técnicas:

Fuzzy Matching com Levenshtein (distância de edição).

TF-IDF + KMeans/Agglomerative Clustering para comparar textos.

Modelos como Sentence-BERT: usam embeddings semânticos para comparar significado das frases.

📊 6. Enriquecimento de dados
Ferramentas: APIs externas, embeddings semânticos, web scraping com IA.

📌 Objetivo: adicionar contexto e informações adicionais úteis a partir de fontes externas.

🔧 Técnicas:

APIs de geolocalização: completar endereços, CEPs, coordenadas.

Modelos de NLP: classificar textos, extrair categorias, prever sentimento.

Web scraping com IA: extrair dados de sites automaticamente.

Embeddings: transformar texto em vetores para inferir similaridades e contextos.